# 라이브러리

In [ ]:
# 필요한 라이브러리 import
import pandas as pd 

from copy import deepcopy

import warnings
warnings.filterwarnings('ignore')

# 아파트 실거래가격지수

데이터 불러오기

In [ ]:
# 아파트 실거래가격지수 데이터 불러오기
apt = pd.read_csv('/content/drive/MyDrive/멀티캠퍼스/data/데이터 시각화PJT/한국부동산원_아파트 지역별 매매 실거래가격지수(월별)_060101210630(https:  www.data.go.kr data 15044283 fileData.do).csv', encoding='cp949')

인덱스 재정의 & 컬럼명 변경
  - 불필요한 인덱스 제거 및 인덱스 재정의

In [ ]:
# 불필요한 인덱스 제거
apt.drop(index=[3,4,5,7,8,9,10,11],inplace=True)

In [ ]:
# 인덱스 재정의
apt.reset_index(drop=True, inplace=True)

컬럼명 변경
- `지역` → `행정구역`
행정구역 컬럼 값 변경
- `서울` → `서울특별시`,... 등

In [ ]:
apt.rename(columns={'지역':'행정구역'}, inplace=True)
apt['행정구역'] = ['전국','수도권','지방','서울특별시','부산광역시','대구광역시','인천광역시','광주광역시','대전광역시','울산광역시','세종특별자치시','경기도','강원도','충청북도','충청남도','전라북도','전라남도','경상북도','경상남도','제주특별자치도']

NameError: ignored

아파트 실거래가격지수 평균 구하기

세종시 값 수정
- 세종시 2012년 07월에 생성 → 12년 06월까지 값 들을 0으로 초기화

In [ ]:
# 컬럼명 확인
apt.columns   

Index(['행정구역', '2006-01', '2006-02', '2006-03', '2006-04', '2006-05',
       '2006-06', '2006-07', '2006-08', '2006-09',
       ...
       '2020-09', '2020-10', '2020-11', '2020-12', '2021-01', '2021-02',
       '2021-03', '2021-04', '2021-05', '2021-06'],
      dtype='object', length=187)

In [ ]:
for col in apt.columns[1:]:  # 2006-01 컬럼부터 하나씩 col에 집어넣어줌
  if col == '2012-07' : break   # 20012-06월까지 반복문이 돌아가도록
  apt.at[10,col] = 0          # 2012-06월까지의 값을 0으로 초기화 

평균구하기

In [ ]:
# 아파트 실거래가격지수를 저장할 변수 생성

# deepcopy를 이용해서 깊은 복사
from copy import deepcopy
apt_avg = deepcopy(apt)

2006년 ~ 2020년

In [ ]:
for i in range(15):           # 2006년부터 2020년까지 (총 15번)
  apt_avg[f'{i+2006}'] = 0.0    # 컬럼생성 및 초기화
  for j in range(len(apt_avg)):           # apt행의 길이 만큼 반복 
    apt_avg[f'{i+2006}'][j] = round(sum(apt_avg.iloc[j, (i*12)+1 : ((i+1)*12)+1 ]) /12, 2) # 각 연도 평균 (round함수 이용해서 소수점 제한)

In [ ]:
# 2021년 전까지 월별 데이터 삭제
for col in apt.columns[1:] :    # 이전과 같은 방법
  if col == '2021-01': break    
  del apt_avg[col]              

2021년도 평균

In [ ]:
apt_avg['2021'] = 0.0    # 컬럼 생성
for n in range(len(apt_avg)):      # apt길이 만큼 반복  
  apt_avg['2021'][n] = round( sum( apt_avg.iloc[n, 1:7] )/6 ,2) 

In [ ]:
# 2021년도 월별 데이터 삭제
for col in apt_avg.columns[1:7] :
  del apt_avg[col]

In [ ]:
# 완성본 csv파일로 저장
pd.DataFrame(apt_avg).to_csv('/content/drive/MyDrive/멀티캠퍼스/[D9&10] 데이터시각화 프로젝트_4조/이경희/data/년도별_아파트_실거래가격지수_평균.csv',index=False)

# 아파트 매매 실거래 평균가격 
- 단위 : 만원/m^2
- 출처 → [아파트 매매 실거래 평균 가격](https://kosis.kr/statHtml/statHtml.do?orgId=408&tblId=DT_KAB_11672_S15&conn_path=I2)

In [ ]:
# 데이터 불러오기
apt_price = pd.read_csv('/content/drive/MyDrive/멀티캠퍼스/data/데이터 시각화PJT/아파트_매매_실거래_평균가격(https:  kosis.kr statHtml statHtml.do?orgId=408&tblId=DT_KAB_11672_S15&conn_path=I2).csv', encoding='cp949')

# 컬럼명 변경
apt_price['행정구역'] = ['전국','서울특별시','부산광역시','대구광역시','인천광역시','광주광역시','대전광역시','울산광역시','세종특별자치시','경기도','강원도','충청북도','충청남도','전라북도','전라남도','경상북도','경상남도','제주특별자치도']
# 세종시 '-'를 데이터 0으로 초기화
for col in apt_price.columns[1:]:  
  apt_price[col].replace('-',0, inplace=True)

# ------------ 년도별 평균 구하기 ----------
apt_price_avg = deepcopy(apt_price) 

# 컬럼 타입 변경
for col in apt_price_avg.columns[1:] :
  apt_price_avg = apt_price_avg.astype({col:'float'})

# 2020년까지 각 연도별 평균 컬럼 생성 및 값 초기화
for i in range(15):         
  apt_price_avg[f'{i+2006}'] = 0.0   
  for j in range(len(apt_price_avg)):          
    apt_price_avg[f'{i+2006}'][j] = round(sum(apt_price_avg.iloc[j, (i*12)+1 : ((i+1)*12)+1 ]) /12, 2) # 각 연도 평균 (round함수 이용해서 소수점 제한)
# 2020년 전까지 월별 데이터 삭제
for col in apt_price.columns[1:] :   
  if col == '2021. 01': break    
  del apt_price_avg[col]            

# 2021년도 평균 컬럼 생성 및 값 초기화
apt_price_avg['2021'] = 0.0    
for n in range(len(apt_price_avg)):      
  apt_price_avg['2021'][n] = round( sum( apt_price_avg.iloc[n, 1:11] )/10 ,2) 
# 2021년도 월별 데이터 삭제
for col in apt_price_avg.columns[1:11] :
  del apt_price_avg[col]  


# 수도권, 지방 행 추가하기
apt_price_avg.loc[18] = apt_price_avg.loc[0]
apt_price_avg.loc[19] = apt_price_avg.loc[0]    
apt_price_avg.loc[18, '행정구역'] = '수도권'  # 1,4,9
apt_price_avg.loc[19, '행정구역'] = '지방'    # 0,2,3,5,6,7,8,10,11,12,13,14,15,16,17

# 전국, 수도권, 지방 컬럼값 재정의
for i in range(1, len(apt_price_avg.columns)):
  apt_price_avg.iloc[17, i] = round ( apt_price_avg.iloc[:,i].mean(), 2)
  apt_price_avg.iloc[18, i] = round ( apt_price_avg.iloc[[1,4,9], i].mean(), 2 )
  apt_price_avg.iloc[19, i] = round ( apt_price_avg.iloc[[0,2,3,5,6,7,8,10,11,12,13,14,15,16,17], i].mean(), 2 )
# 인데스 재정의
apt_price_avg = apt_price_avg.reindex(index=[ 0, 18, 19, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17])
apt_price_avg.reset_index(drop=True, inplace = True) 

# 전처리 완료

In [ ]:
# 완성본 csv 파일로 저장
#pd.DataFrame(apt_price_avg).to_csv('/content/drive/MyDrive/멀티캠퍼스/[D9&10] 데이터시각화 프로젝트_4조/이경희/data/아파트 매매 실거래 평균 가격.csv', index=False)
apt_price_avg.head()

,행정구역,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,전국,252.31,224.62,236.70,276.06,241.62,266.08,265.71,287.68,310.00,343.58,368.53,406.08,409.67,428.16,454.28,483.37
1,수도권,299.93,313.70,380.34,418.34,400.23,392.18,377.99,380.51,396.76,423.69,456.48,508.64,531.75,603.75,667.89,825.25
2,지방,127.22,128.90,135.42,148.15,155.67,180.30,197.59,213.06,227.06,243.02,255.70,271.41,276.45,287.12,326.14,348.96
3,서울특별시,442.73,450.44,551.06,633.17,609.23,586.43,567.72,556.91,574.17,610.86,663.33,765.80,806.94,1000.13,1104.96,1379.94
4,부산광역시,150.10,157.31,164.43,183.34,208.66,255.72,258.63,263.15,277.16,299.27,330.47,352.92,351.37,372.90,416.99,422.24
5,대구광역시,167.63,165.33,166.70,171.31,173.05,199.52,221.70,246.77,282.48,323.48,321.24,340.03,360.99,355.84,398.37,391.85
6,인천광역시,184.33,216.57,276.60,284.82,277.23,272.38,260.43,264.02,277.45,300.30,321.63,339.26,350.28,360.65,398.90,485.77
7,광주광역시,113.89,117.81,123.62,129.87,135.94,167.55,182.49,193.52,211.93,234.91,249.17,265.14,281.42,286.52,326.25,349.46
8,대전광역시,173.09,161.23,161.91,180.75,202.78,239.52,236.97,244.12,250.55,254.95,265.09,278.39,297.34,316.24,376.57,429.57
9,울산광역시,138.79,164.67,169.50,181.42,191.32,216.70,245.53,251.58,268.30,289.72,311.86,313.82,295.26,306.58,346.53,339.52
